### This program solves the stochastic growth model with linear quadratic dynamic programming.

#### Translated from Eva Carceles-Poveda's (2003) MATLAB codes

In [1]:
# Importing packages
import numpy as np

# needed for compact printing of numpy arrays
# use precision to set the number of decimal digits to display
# use suppress=True to show values in full decimals instead of using scientific notation
np.set_printoptions(suppress=True,precision=4,linewidth=np.inf)

#### The riccati1 (Ricatti difference equation) function is copied below

In [2]:
def riccati1(U,DJ,DH,S,C,B,Sigma,beta):
    """ Syntax: [P1,J,d] = riccati(U,DJ,DH,S,C,B,Sigma,beta)
    
    This function solves for the value and policy functions of a linear quadratic problem 
    by iterating on the Ricatti difference equation. The inputs are the return function at 
    steady state (U), the Jacobian and Hessian matrices (DJ and DH), the vector of states 
    (S=[z; s]), the vector of controls (C=[x]), the matrix B satisfying [1;z';s']=B[1;z;s;x] 
    the variance covariance matrix of [1 z' s'] (Sigma) and the discount factor beta.
    
    Translated from Eva Carceles-Poveda 2003 MATLAB code
    """

    tolv = 1e-07

    ns1, ns2 = S.shape
    if ns2 > ns1:
        S = S.T

    ns = max(S.shape)

    nc1, nc2 = C.shape
    if nc2 > nc1:
        C = C.T
    
    nc = max(C.shape)

    WW = np.concatenate((S.T,C.T),axis=1).T
    Q11 = U - WW.T@DJ + 0.5*WW.T@DH@WW
    Q12 = 0.5*(DJ-DH@WW)
    Q22 = 0.5*DH
        
    QQ = np.concatenate((np.concatenate((Q11, Q12.T),axis=1),np.concatenate((Q12,Q22),axis=1)))

    nq=ns+nc+1

    # Partition Q to separate states and controls 
    Qff = QQ[0:ns+1,0:ns+1]
    Qfx = QQ[ns+1:nq,0:ns+1]
    Qxx = QQ[nq-nc:nq,nq-nc:nq]
    
    # Initialize matrices
    P0 = -0.1*np.eye((ns+1))
    P1 = np.ones((ns+1,ns+1))
    
    # Iterate on Bellman's equation until convergence
    while np.linalg.norm(np.abs(P1-P0)) > tolv:
        P1 = P0.copy()
        M = B.T@P0@B;
        Mff = M[0:ns+1,0:ns+1]
        Mfx = M[ns+1:nq,0:ns+1]
        Mxx = M[nq-nc:nq,nq-nc:nq]

        P0=Qff+beta*Mff-(Qfx+beta*Mfx).T@np.linalg.inv(Qxx+beta*Mxx)@(Qfx+beta*Mfx)   

      

    J = -np.linalg.inv(Qxx + beta*Mxx)@(Qfx + beta*Mfx)

    d = beta/(1-beta)*np.trace(P0@Sigma)

    return P1,J,d

#### The Hodrick-Prescott filter function is copied below

In [3]:
def hp1(y,w):

    """ Syntax: yhp, ytr = hp1(y, w)
    
    This function computes the filtered series of y, using
    a smoothing parameter w. 
    
    The code is from I. Izvorski.
    """

    t, s = y.shape
    
    if t < s:
        y = y.T

    a = 6*w + 1
    b = -4*w
    c = w
    d = np.array([[c,b,a]])
    d = np.ones((t,1))*d
    m = np.diag(d[:,2])+np.diag(d[0:-1,1],1)+np.diag(d[0:-1,1],-1)
    m = m+np.diag(d[0:-2,0],2)+np.diag(d[0:-2,0],-2)

    m[0,0] = 1+w;       m[0,1] = -2*w;
    m[1,0] = -2*w;      m[1,1] = 5*w+1;
    m[-2,-2] = 5*w+1;   m[-2,-1] = -2*w;
    m[-1,-2] = -2*w;    m[-1,-1] = 1+w;
    
    ytr = np.matmul(np.linalg.inv(m),y)
    yhp = y-ytr

    return yhp, ytr

In [4]:
# Model Parameters
alf = 0.33
beta = 0.96
rho = 0.95
delta = 0.10
tolv = 1e-07
sigmae = 0.007

In [5]:
# Steady state
zs = 0
ks = (np.exp(zs)*alf*beta/(1-beta*(1-delta)))**(1/(1-alf))
ins = delta*ks
cs = np.exp(zs)*ks**alf-ins

In [6]:
# Construct the quadratic expansion of the utility function
R = np.log(cs)

DJ = np.zeros((3,1))
DJ[0,0] = (np.exp(zs)*ks**alf)/cs #Jz
DJ[1,0] = (np.exp(zs)*alf*ks**(alf-1))/cs #Jk
DJ[2,0] = (-1)/cs #Jx

Hzz = ((np.exp(zs)*ks**alf)*cs - (np.exp(zs)*ks**alf)**2 )/(cs**2)
Hkk = (((np.exp(zs)*alf*(alf-1)*ks**(alf-2))*cs)-(np.exp(zs)*alf*ks**(alf-1))**2 )/(cs**2)
Hxx = (-1)/(cs**2)
Hzk = (((np.exp(zs)*alf*ks**(alf-1))*cs) - (np.exp(zs)*ks**alf*np.exp(zs)*alf*ks**(alf-1)))/(cs**2)
Hzx = (np.exp(zs)*ks**alf)/(cs**2)
Hkx = (np.exp(zs)*alf*ks**(alf-1))/(cs**2)

DH = np.array([[Hzz, Hzk, Hzx],
    [Hzk, Hkk, Hkx],
    [Hzx, Hkx, Hxx]])

S = np.array([[zs, ks]])
C = np.array([[ins]])

In [7]:
# Define input matrix B
B = np.array([[1, 0, 0, 0],
    [0, rho, 0, 0],
    [0, 0, 1-delta, 1]])

# Define the variance covariance matrix
Sigma = np.array([[0,0,0],[0,sigmae**2,0],[0,0,0]])

In [8]:
P,J,d = riccati1(R,DJ,DH,S,C,B,Sigma,beta)

In [9]:
print(' The optimal value function is [1 z s]P0[1; z; s]+d, where P and d are given by:')
print(P)
print(round(d,4))

print(' The policy function is x=J[1; z; s] where J is:')
print(J)

 The optimal value function is [1 z s]P0[1; z; s]+d, where P and d are given by:
[[-0.4025  8.0839  0.7369]
 [ 8.0839  1.0029 -0.1915]
 [ 0.7369 -0.1915 -0.0819]]
0.0012
 The policy function is x=J[1; z; s] where J is:
[[ 0.4983  0.8607 -0.0411]]


In [10]:
# Simulation of the model
T = 115
N = 100
ss = np.zeros((N,4))
cc = np.zeros((N,4))
rng = np.random.Generator(np.random.MT19937())

for j in np.arange(0,N):
    r = rng.standard_normal((T+1,1))
    z = np.ones((T+1,1))
    z[0] = 0
    k = np.zeros((T+1,1))
    k[0] = ks
    i = np.zeros((T,1))
    c = np.zeros((T,1))
    y = np.zeros((T,1))

    for t in np.arange(0,T):
        y[t] = np.exp(z[t])*k[t]**alf
        i[t] = J@np.array([[1],z[t],k[t]])
        c[t] = y[t]-i[t]
        k[t+1] = (1-delta)*k[t] + i[t]
        z[t+1] = rho*z[t]+sigmae*r[t]


    z = z[0:T]
    k = np.log(k[0:T])
    y = np.log(y[0:T])
    c = np.log(c)
    i = np.log(i)

    dhp, dtr = hp1(np.concatenate((y, c, i, k),axis=1),1600)
    ss[j,:] = np.std(dhp,axis=0,ddof=1)*100
    Corr = np.corrcoef(dhp,rowvar=False)
    cc[j,:] = Corr[:,0]

stdv = np.mean(ss,axis=0)
corr = np.mean(cc,axis=0)

print('std(x) std(x)/std(y) corr(x,y) for y, c, i, k:')
print(np.concatenate((stdv[:,np.newaxis], (stdv/stdv[0])[:,np.newaxis], \
                      corr[:,np.newaxis]),axis=1))



std(x) std(x)/std(y) corr(x,y) for y, c, i, k:
[[0.9634 1.     1.    ]
 [0.6802 0.706  0.9526]
 [2.1185 2.199  0.9468]
 [0.629  0.6529 0.4391]]


In [11]:
#!jupyter nbconvert --to script growthLQDP.ipynb